# Quantum State Tomography via Compressed Sensing
The idea is to replicate the state reconstruction described in the paper "Quantum State Tomography via Compressed Sensing (2010)" by D.Gross, Y. Liu, et al. There should already be libreries that would help me integrate teh code better.

The main idea is to make "Matrix recovery using Pauli measurements". Consider an unkonw quantum state $\rho$ form by $n$ qubits. An $n$-qubit Pauli matrix is of the form $P = \bigotimes_{i=1}^N P_i$, where $P_i \in \{ 1, X, Y, Z \}$, thus $P \in \{ 1, X, Y, Z \}^{\otimes n}$. There are $d^2$ such matrices, where $d=2^n$ (the whole dimension of the Hilbert space). The protocol proceeds as follows: (1) measure the expectation value $\text{tr}(\rho P)$ with random Pauli, (2) then solve the convex optimization problem:
$$
min \{ ||\sigma||_{\text{tr}}\} \quad \text{s.t} \quad \text{tr}(\sigma) = 1, \quad \text{tr}(P \sigma) = \text{tr}(P \rho).
$$

The necessary number of mesurements goes as $N = \mathcal{O}(dr\log^2{d})$.

Dudo en si aplicar SVT o SDP.


In [1]:
pip install cvxpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
import numpy as np
import math
import qibo
import matplotlib.pyplot as plt #(pip install --force pillow) in case it does not work
from itertools import product
from qibo.models import QFT
from qibo.models import Circuit
from qibo import gates
import itertools
import random

from qibo.quantum_info import random_unitary
from qibo.quantum_info import fidelity
import qibo.quantum_info
from qibo.quantum_info import to_choi
import cvxpy as cp
from qibo.quantum_info import diamond_norm

In [2]:
def pseudorandom_circuit(nqubits, density_matrix=None ):
    c = Circuit(nqubits, density_matrix=density_matrix)
    for i in range(nqubits):
        haar_U = random_unitary(2, measure='haar')
        c.add(gates.Unitary(haar_U, i))
    for i in range(nqubits-1):
        c.add(gates.CNOT(i,i+1))
    for i in range(nqubits):
        haar_U = random_unitary(2, measure='haar')
        c.add(gates.Unitary(haar_U, i))
    return c

def Pauli_matrices(nqubits):
    X = np.matrix([[0,1],[1,0]])    
    Y = np.matrix([[0,-1j],[1j,0]]) 
    Z = np.matrix([[1,0],[0,-1]])
    I = np.eye(2)
    Paulis = [I,X,Y,Z]
    combinations = product(Paulis, repeat=nqubits)
    combinations = list(combinations)
    all_paulis = []
    for paulis in combinations:
            p = np.kron(paulis[0], paulis[1])
            for i in range(2, nqubits):
                p = np.kron(p, paulis[i])
            all_paulis.append(p)
    return all_paulis

I find some similarities of this problem with Semidefinite Programming (SDP), so first I am going to implement that.

In [3]:
#Let's work first with the ideal values.
nqubits=4

c = pseudorandom_circuit(nqubits, density_matrix=True)
density = c()
density = np.array(density)
ideal_val = []
Paulis = Pauli_matrices(nqubits)
for P in Paulis:
    ideal_pauli_trace_val = np.trace(P*density)
    ideal_val.append(ideal_pauli_trace_val.real)



[Qibo 0.1.11|INFO|2024-04-15 09:35:12]: Using numpy backend on /CPU:0


In [4]:
X = cp.Variable(shape=(2**nqubits, 2**nqubits), hermitian = True)   #Fuck, hay que especificar que es hermítica
tau = 5
delta = 0.1
cost = cp.norm(X,'nuc')
constraints = [
    cp.trace(Paulis[i] @ X) == ideal_val[i] for i in range(len(ideal_val))
]
constraints += [cp.trace(X) == 1]
constraints += [X >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution = X.value
print('Fidelity: ', fidelity(density, solution))

Fidelity:  0.999999995674867


In [5]:
val, vec = np.linalg.eigh(solution)
print(val)

[8.31059210e-11 1.12216488e-10 1.16816144e-10 1.23349828e-10
 1.30017165e-10 1.37481575e-10 1.50909083e-10 1.60646115e-10
 1.73073471e-10 1.90239801e-10 2.14239985e-10 2.46198226e-10
 2.66115512e-10 4.25037701e-10 6.15182765e-10 9.99999996e-01]


Okay, so it seems it does work nice. Convex optimization problems works in poly time to get results. Only problem is that the computational time increases with the dimension of the problem, as we need to get more data, thus, the algorithm for post-processing is not as effective as we would like. Let's see know what happens when we introduce measurements.

In [6]:
def chose_random_Pauli(nqubits):
    X = np.matrix([[0,1],[1,0]])    
    Y = np.matrix([[0,-1j],[1j,0]]) 
    Z = np.matrix([[1,0],[0,-1]])
    I = np.eye(2)
    Paulis = [I,X,Y,Z]

    if nqubits==1:
        chosen = random.choice(Paulis)

    elif nqubits > 1:
        chosen0 = random.choice(Paulis)
        chosen1 = random.choice(Paulis)
        chosen = np.kron(chosen0, chosen1)
        for _ in range(2,nqubits):
            choseni = random.choice(Paulis)
            chosen = np.kron(chosen, choseni)
    return chosen

def random_pauli_basis(nqubits:int):
    """Choses a random Pauli basis to measure with.

        It returns a list of qibo Pauli gates that can 
        be used during the execution of the circuit. These
        gates are randomly chossen among the Pauli basis.

        Attributes:
            nqubits(int): Number of qubits of the quantum
                          system  
    """
    gate_list = [gates.X, gates.Y, gates.Z]
    basis = []
    for _ in range(nqubits):
        basis.append(random.choice(gate_list))

    return basis

def qibo_to_matrix(lista):
    """Given a list that contains a set of Pauli basis 
        represented with qibo, it returns a list that
        transcribes the qibo classes to a string. Useful
        to work with.

        Attributes:
            lista(list): list that contains the Pauli basis.
    """
    strings = []
    for instance in lista:
        if instance is qibo.gates.X:
            strings.append('X')
        elif instance is qibo.gates.Y:
            strings.append('Y')
        elif instance is qibo.gates.Z:
            strings.append('Z')
        else: print('Bro there has been a mistake and one of the gates is not a Pauli.')

    return strings


In [57]:
#Let's take a look at the number of the Paulis we need to take
nqubits = 4
r = 3   #rank of rho. For a pure state it should be 1.
d = 2**nqubits  #dimension of the hilber space
print('Number of Paulis measured (m): ', r*d*np.log2(d)**2)

Number of Paulis measured (m):  768.0


In [121]:
#Let's work first with measurements.
nqubits=2
m = 1    #number of Paulis we chose, order of rdlog^2d   

all_Paulis_chosen=[]

for _ in range(m):
    c = pseudorandom_circuit(nqubits, density_matrix=False)
    P = chose_random_Pauli(nqubits)
    all_Paulis_chosen.append(P)
    c.add(gates.Unitary(P, *range(nqubits)))
    c.add(gates.M(*range(nqubits)))
    final = c(nshots=1000)
    print(final.frequencies(binary=True))
    print(final.samples(binary=True))
    print(final)

Counter({'10': 427, '11': 371, '00': 179, '01': 23})
[[1 1]
 [1 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [1 1]]
(0.40964+0.08305j)|00> + (-0.09469+0.10889j)|01> + (-0.06842-0.65319j)|10> + (-0.18743+0.58138j)|11>


In [130]:
for observ in all_Paulis_chosen:
    print(observ)
    val, vec = np.linalg.eigh(observ)
    suma=0
    for i in range(len(vec)):
        suma += val[i]*np.outer(vec[:,i], np.conjugate(vec[:,i]))
    print(suma)

print(val, vec)


[[0.+0.j 0.+0.j 0.-1.j 0.-0.j]
 [0.+0.j 0.+0.j 0.-0.j 0.-1.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.-1.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.-1.j]
 [0.+1.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+1.j 0.+0.j 0.+0.j]]
[-1. -1.  1.  1.] [[-0.70710678-0.j         -0.        -0.j          0.        +0.j
  -0.70710678+0.j        ]
 [ 0.        +0.j          0.        -0.70710678j  0.        -0.70710678j
   0.        +0.j        ]
 [ 0.        +0.70710678j  0.        +0.j          0.        +0.j
   0.        -0.70710678j]
 [ 0.        +0.j         -0.70710678+0.j          0.70710678+0.j
   0.        +0.j        ]]


I found a problem on how to measure the expectation value of a pauli operator (basically if the identity is on it). So, I am leaving this issue at the moment and see what happens next. Let's now move on an see what happens when we use random unitaries (following the distribution given by Haar) and the clifford group.

In [272]:
#Let's work with the ideal values.
nqubits=4

c = pseudorandom_circuit(nqubits, density_matrix=True)
density = c()
density = np.array(density)
ideal_val = []
Random_Unitaries = []
for _ in range(1000):
    Random_Unitaries.append(random_unitary(dims=2**nqubits, measure='haar'))
for U in Random_Unitaries:
    ideal_pauli_trace_val = np.trace(U*density)
    ideal_val.append(ideal_pauli_trace_val.real)

In [274]:
X = cp.Variable(shape=(2**nqubits, 2**nqubits), hermitian = True)   #Fuck, hay que especificar que es hermítica
cost = cp.norm(X,'nuc')
constraints = [
    cp.trace(Random_Unitaries[i] @ X) == ideal_val[i] for i in range(len(ideal_val))
]
constraints += [cp.trace(X) == 1]
constraints += [X >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution = X.value
print(X.value)
#print('Fidelity: ', fidelity(density, solution))

X_lasso = cp.Variable(shape=((2**nqubits), (2**nqubits)), hermitian = True)
vec=[]
mu = 0.4
norm = np.sqrt(2**nqubits/m)
for i in range(len(Random_Unitaries)):
        vec.append(cp.trace(Random_Unitaries[i]@ X_lasso) - ideal_val[i])   #funciona igual con que sin norm
vec = cp.vstack(vec)
cost = 0.5*cp.norm(vec,2)**2 + mu*cp.norm(X_lasso, 'nuc')
constraints = []

constraints += [X_lasso >> 0]
#constraints += [cp.trace(X_lasso) == 1]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_lasso = X_lasso.value
print('Done!!')

#print('Fidelity of the replied DS quantum state: ', fidelity(density, solution_estimated/np.trace(solution_estimated)))
print('Fidelity of the replied Lasso quantum state: ', fidelity(density, solution_lasso/np.trace(solution_lasso)))
#print('Trace of the DS: ',np.trace(solution_estimated))
print('Trace of the Lasso: ',np.trace(solution_lasso))

None


c:\Users\PW0732YT\anaconda3\envs\VirtualEnv_qibo\lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "
c:\Users\PW0732YT\anaconda3\envs\VirtualEnv_qibo\lib\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "


Done!!
Fidelity of the replied Lasso quantum state:  0.09428615005726895
Trace of the Lasso:  (0.38564697169559814+0j)


It seems that the Haar distribution is not enaugh to characterize the quantum state. Let's see now if the Clifford group is more lucky.

In [276]:
#let's try by choosing random clifford gates
H = 1/np.sqrt(2)*np.array([[1,1],[1,-1]], dtype=complex)
S = np.array([[1,0],[0,1j]])

single_qubit_cliffords = [
    '',
    'H', 'S',
    'HS', 'SH', 'SS',
    'HSH', 'HSS', 'SHS', 'SSH', 'SSS',
    'HSHS', 'HSSH', 'HSSS', 'SHSS', 'SSHS',
    'HSHSS', 'HSSHS', 'SHSSH', 'SHSSS', 'SSHSS',
    'HSHSSH', 'HSHSSS', 'HSSHSS'
]

clifford_gates = []

for string in single_qubit_cliffords:
    product = np.eye(2)
    for symbol in string:
        if symbol == 'H':
            product = product @ H

        if symbol == 'S':
            product = product @ S

        else: 
            pass
    clifford_gates.append(product)

In [277]:
#Let's work with the ideal values.
nqubits=2

c = pseudorandom_circuit(nqubits, density_matrix=True)
density = c()
density = np.array(density)
ideal_val = []
gates_C = []
for _ in range(1000):
    C_0 = random.choice(clifford_gates)
    C_1 = random.choice(clifford_gates)
    C = np.kron(C_0, C_1)
    for _ in range(2, nqubits):
        C_i = random.choice(clifford_gates)
        C = np.kron(C, C_i)
    gates_C.append(C)

for C in gates_C:
    ideal_pauli_trace_val = np.trace(C*density)
    ideal_val.append(ideal_pauli_trace_val.real)

In [278]:
X = cp.Variable(shape=(2**nqubits, 2**nqubits), hermitian = True)   #Fuck, hay que especificar que es hermítica
cost = cp.norm(X,'nuc')
constraints = [
    cp.trace(gates_C[i] @ X) == ideal_val[i] for i in range(len(ideal_val))
]
constraints += [cp.trace(X) == 1]
constraints += [X >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution = X.value
print(X.value)
#print('Fidelity: ', fidelity(density, solution))

X_lasso = cp.Variable(shape=((2**nqubits), (2**nqubits)), hermitian = True)
vec=[]
mu = 0.4
norm = np.sqrt(2**nqubits/m)
for i in range(len(Random_Unitaries)):
        vec.append(cp.trace(gates_C[i]@ X_lasso) - ideal_val[i])   #funciona igual con que sin norm
vec = cp.vstack(vec)
cost = 0.5*cp.norm(vec,2)**2 + mu*cp.norm(X_lasso, 'nuc')
constraints = []

constraints += [X_lasso >> 0]
#constraints += [cp.trace(X_lasso) == 1]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_lasso = X_lasso.value
print('Done!!')

#print('Fidelity of the replied DS quantum state: ', fidelity(density, solution_estimated/np.trace(solution_estimated)))
print('Fidelity of the replied Lasso quantum state: ', fidelity(density, solution_lasso/np.trace(solution_lasso)))
#print('Trace of the DS: ',np.trace(solution_estimated))
print('Trace of the Lasso: ',np.trace(solution_lasso))

None


c:\Users\PW0732YT\anaconda3\envs\VirtualEnv_qibo\lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "
c:\Users\PW0732YT\anaconda3\envs\VirtualEnv_qibo\lib\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "


Done!!
Fidelity of the replied Lasso quantum state:  0.2470693043703052
Trace of the Lasso:  (0.5306450316891889+0j)


The Clifford group does not seem to work neither. Thus, we are going to try to use the method for quantum process.

In [8]:
def defined_map(choi, rho):
    """
    Given a choi representation of the quantum map, it returns the quantum state that
    passes through this channel.
    Atributes:
    - choi: isomorphism of the channel.
    - rho: initial quantum state.
    """
    dim_rho = rho.shape
    nqubits = np.log2(dim_rho[0])
    p = choi@np.kron(np.eye(dim_rho[0]), np.transpose(rho))
    reshaped_p = p.reshape(int(2**nqubits),int(2**nqubits),int(2**nqubits),int(2**nqubits))
    predicted = np.trace(reshaped_p, axis1=1, axis2=3)
    return predicted

In [9]:
#Let's work first with the ideal values.
nqubits=3
m = 2**nqubits*nqubits**2
print(m)
Paulis_A = Pauli_matrices(nqubits)
Paulis_B = Pauli_matrices(nqubits)
Paulis_B_conj = np.conjugate(Paulis_B)
val_B=[]
vec_B=[]
for PB in Paulis_B_conj:
    val, vec = np.linalg.eigh(PB)
    val_B.append(val)
    vec_B0 = []
    for i in range(len(vec[0,:])):
        vec_B0.append(vec[:,i])
    vec_B.append(vec_B0)
vec_B = np.array(vec_B)

c = pseudorandom_circuit(nqubits, density_matrix=True)
randoms_PA = []
selected_PB = Paulis_B
A_list = []

for k in range(m):
    #PA = Paulis_A[k]
    PA = chose_random_Pauli(nqubits)
    randoms_PA.append(PA) 
    for j in range(vec_B.shape[0]):
        trace_val=0
        for i in range(vec_B.shape[1]):
            copi = c.copy()
            density = copi(initial_state=np.outer(vec_B[j,i], np.conjugate(vec_B[j,i])))
            density = np.array(density)
            trace_val += val_B[j][i]*np.trace(PA @ density)     #posible fuente de errores si no estoy cogiendo bien el eigenvalue
        trace = 1/(2**nqubits)*trace_val
        A_list.append(trace.real)

print('Ideal trace values', A_list)

#here we are trying to compute the Choi isomorphism
import qibo.quantum_info 
copi2 = c.copy()
unit_matrix = copi2.unitary()
choi = qibo.quantum_info.to_choi(unit_matrix, order='row')
#print(choi)
#print(choi.shape)


    
#let's check that everything is correct
copi3 = c.copy()
final_dens = copi3()
final_dens = np.array(final_dens)
initial = np.zeros(shape=(2**nqubits, 2**nqubits))
initial[0,0]=1
reconstrucion = defined_map(choi, initial)
print('Check the computed choi is the correct one: ', fidelity(final_dens, reconstrucion))

#convex opt problem

from qibo.quantum_info import diamond_norm
X = cp.Variable(shape=((2**nqubits)**2, (2**nqubits)**2), hermitian = True)   #Fuck, hay que especificar que es hermítica
cost = cp.norm(X,'nuc')
constraints = []
for i in range(len(randoms_PA)):
    for j in range(len(selected_PB)):
        constraints += [cp.trace(cp.kron(randoms_PA[i], selected_PB[j]) @ X) == A_list[len(selected_PB)*i + j]]

constraints += [X >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution = X.value
#print('Diam norm of choi: ', diamond_norm(choi))
#print('Diam norm of solution: ', diamond_norm(solution))
#print('Diamond norm: ', diamond_norm(choi, solution))

reconstrucion2 = defined_map(2**nqubits*solution, initial)   #por algun motivo sale un factor 4, que entiendo que debe de ser 2**nqubits
print('Fidelity of the replied quantum state: ', fidelity(final_dens, reconstrucion2))
print('Diamond norm btw ideal choi and estimated channel: ', diamond_norm(choi, 2**nqubits*solution))   #pasa algo con la diamond norm, al parecer se van sumando los resultados si no se va reiniciando
    

72
Ideal trace values [-6.938893903907228e-18, -3.122502256758253e-17, -4.85722573273506e-17, 1.3877787807814457e-17, 2.0816681711721685e-17, 0.004215478914276254, 0.012953673083711953, -0.022449631069365573, 0.0, -0.0045952227344125685, -0.01412058137613581, 0.02447196562168942, 3.469446951953614e-17, -0.00012391885748271395, -0.00038078813852008425, 0.0006599327596213053, 3.469446951953614e-17, 0.0007911877832561393, 0.0039978992037009095, 0.0024553951266996013, 1.0408340855860843e-17, 0.04569329546434728, 0.1986621098354634, 0.0046216239376628936, 0.0, 0.040897235000575086, 0.17622877808097315, -0.0025948615504514244, 8.326672684688674e-17, 0.03782490897437005, 0.22309314452352208, 0.2534426614819695, 2.42861286636753e-17, 0.0024313305668235453, 0.012285597354694436, 0.007545461837903413, 0.0, 0.03258585006533292, 0.0802573385351803, -0.25813971864976837, -5.551115123125783e-17, 0.03152948897940841, 0.07963694942632064, -0.24133723648729472, -3.469446951953614e-17, -0.06068536186643

It works correctly at a theoretical level. Let's try it know by computing real measurements.

In [11]:
def unitary_transformation_to_computational_basis(matrix):
    """
    Computes the unitary transformation U need to go from the computational basis,
    aka the kroneker product of severa Pauli Z matrices, to a desired matrix. That 
    is P = U^dagger Z U.
    Attributes:
    - matrix: array that wants to be decomposed. Acts as the previous P.
    """
    nqubits = int(np.log2(matrix.shape[0]))
    Z = np.matrix([[1,0],[0,-1]])
    if nqubits == 1:
        O = Z
    else:
        O = np.kron(Z,Z)
        for _ in range(2,nqubits):
            O = np.kron(O, Z)
    val_A, vec_A = np.linalg.eigh(matrix)
    val_z, vec_z = np.linalg.eigh(O)
    u1 = vec_A.conj().T
    u2 = vec_z.conj().T
    u = u2.conj().T@u1
    
    return u

def exp_val_in_Z(freq):
    """
    Given the dictionary that contains the measured bits and counts,
    it returns the expected value of the Z operator.
    Attributes:
    - freq: dictionary containing bits and counts.
    """
    expected_value=0
    count=0
    for bit in freq:
        expected_value+=(-1)**(bit.count('1'))*freq[bit]
        count += freq[bit]
        
    exp_val = expected_value/count
    return exp_val

In [269]:
#Let's work first with the ideal values.
nqubits=3
m = 2**nqubits*nqubits**2
print(m)
#if we wont to implement all the Paulis in B
Paulis_B = Pauli_matrices(nqubits)
Paulis_B_conj = np.conjugate(Paulis_B)
val_B=[]
vec_B=[]
for PB in Paulis_B_conj:
    val, vec = np.linalg.eigh(PB)
    val_B.append(val)
    vec_B0 = []
    for i in range(len(vec[0,:])):
        vec_B0.append(vec[:,i])
    vec_B.append(vec_B0)
vec_B = np.array(vec_B)

selected_PB = Paulis_B

c = pseudorandom_circuit(nqubits, density_matrix=True)
randoms_PA = []
selected_PB =[]
A_ideal = []
A_estimated = []

for k in range(m):
    PA = chose_random_Pauli(nqubits)
    u = unitary_transformation_to_computational_basis(PA)
    randoms_PA.append(PA) 
    for j in range(vec_B.shape[0]):
        trace_val_ideal=0
        trace_val_estimated=0
        for i in range(vec_B.shape[1]):
            copi_ideal = c.copy()
            density = copi_ideal(initial_state=np.outer(vec_B[j,i], np.conjugate(vec_B[j,i])))
            density = np.array(density)
            trace_val_ideal += val_B[j][i]*np.trace(PA @ density)     #posible fuente de errores si no estoy cogiendo bien el eigenvalue

            copi_estimated = c.copy()
            copi_estimated.add(gates.Unitary(u, *range(nqubits)))
            copi_estimated.add(gates.M(*range(nqubits)))
            result = copi_estimated(initial_state=np.outer(vec_B[j,i], np.conjugate(vec_B[j,i])), nshots=10000)
            freq = result.frequencies(binary=True)
            trace_val_estimated += val_B[j][i]*exp_val_in_Z(freq)
        trace_ideal = 1/(2**nqubits)*trace_val_ideal
        trace_estimated = 1/(2**nqubits)*trace_val_estimated
        A_ideal.append(trace_ideal.real)
        A_estimated.append(trace_estimated.real)

print('Ideal trace values', A_ideal)
print('Estimated trace values', A_estimated)

72
Ideal trace values [1.734723475976807e-17, -1.9081958235744878e-17, -6.245004513516506e-17, -4.119968255444917e-17, -0.0004969731609691334, -0.0033192483768651627, -0.001674585226980281, 0.0029174077083270623, 0.001695460290559404, -0.00040194609526403996, 0.0008737317028425298, -0.0007625968663539824, -0.001801535354183296, 0.0020250903770996868, -0.00026890287695185283, -0.000442156476343266, 0.032995242776163336, 0.17650853063426658, -0.18585792109117324, 0.1273393746016364, -0.11208153232944978, 0.04340000267362053, -0.14860412082230515, -0.1027878419776545, 0.009976287729672226, -0.22417608527565858, -0.3385553339983331, -0.19166269057854585, 0.04030779242398189, 0.026640137601903743, -0.17462164635666633, -0.34764316141422086, 0.029218745315080907, 0.15550878793440534, -0.16690707492319817, 0.11052487107056225, 0.1085672816737272, -0.11377195501224877, -0.06744238391367174, -0.10334037734925405, -0.009380428664103192, 0.03186282516892098, -0.2170890436698595, -0.33747816706901

In [270]:
#here we are trying to compute the Choi isomorphism
import qibo.quantum_info 
copi2 = c.copy()
unit_matrix = copi2.unitary()
choi = qibo.quantum_info.to_choi(unit_matrix, order='row')
#print(choi)
#print(choi.shape)

#let's check that everything is correct
copi3 = c.copy()
final_dens = copi3()
final_dens = np.array(final_dens)
initial = np.zeros(shape=(2**nqubits, 2**nqubits))
initial[0,0]=1
reconstrucion = defined_map(choi, initial)
print('Check te computed choi is the correct one: ', fidelity(final_dens, reconstrucion))

Check te computed choi is the correct one:  0.9999999999999996


In [271]:
#convex opt problem
from qibo.quantum_info import diamond_norm
X_ideal = cp.Variable(shape=((2**nqubits)**2, (2**nqubits)**2), hermitian = True)   #Fuck, hay que especificar que es hermítica
cost = cp.norm(X_ideal,'nuc')
constraints = []
for i in range(len(randoms_PA)):
    for j in range(len(selected_PB)):
        constraints += [cp.trace(cp.kron(randoms_PA[i], selected_PB[j]) @ X_ideal) == A_ideal[len(selected_PB)*i + j]]

constraints += [X_ideal >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_ideal = X_ideal.value
print('Nice')
#print('Diam norm of choi: ', diamond_norm(choi))
#print('Diam norm of solution: ', diamond_norm(solution))
#print('Diamond norm: ', diamond_norm(choi, solution))

X_lasso = cp.Variable(shape=((2**nqubits)**2, (2**nqubits)**2), hermitian = True)
vec=[]
mu = 10
norm = np.sqrt(2**nqubits/m)
for i in range(len(randoms_PA)):
    for j in range(len(selected_PB)):
        vec.append(cp.trace(cp.kron(randoms_PA[i], selected_PB[j])@ X_lasso) - A_estimated[len(selected_PB)*i + j])   #funciona igual con que sin norm
vec = cp.vstack(vec)
cost = 0.5*cp.norm(vec,2)**2 + mu*cp.norm(X_lasso, 'nuc')
constraints = []

constraints += [X_lasso >> 0]
#constraints += [cp.trace(X_lasso) == 1]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_lasso = X_lasso.value
print('Trace: ', np.trace(solution_lasso))
solution_lasso = solution_lasso/np.trace(solution_lasso)
print('Trace: ', np.trace(solution_lasso))

reconstrucion2 = defined_map(2**nqubits*solution_ideal, initial)   #por algun motivo sale un factor 4, que entiendo que debe de ser 2**nqubits
reconstrucion3 = defined_map(2**nqubits*solution_lasso, initial)
print('Fidelity of the replied ideal quantum state: ', fidelity(final_dens, reconstrucion2))
choi = qibo.quantum_info.to_choi(unit_matrix, order='row')
print('Diamond norm btw ideal choi and ideal estimated channel: ', diamond_norm(choi, 2**nqubits*solution_ideal))   #pasa algo con la diamond norm, al parecer se van sumando los resultados si no se va reiniciando
print('Fidelity of the replied lasso estimated quantum state: ', fidelity(final_dens, reconstrucion3))
choi = qibo.quantum_info.to_choi(unit_matrix, order='row')
print('Diamond norm btw ideal choi and approx lasso estimated channel: ', diamond_norm(choi, 2**nqubits*solution_lasso))   #pasa algo con la diamond norm, al parecer se van sumando los resultados si no se va reiniciando
    

Nice


c:\Users\PW0732YT\anaconda3\envs\VirtualEnv_qibo\lib\site-packages\cvxpy\problems\problem.py:158: UserWarning: Objective contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn("Objective contains too many subexpressions. "
c:\Users\PW0732YT\anaconda3\envs\VirtualEnv_qibo\lib\site-packages\cvxpy\problems\problem.py:164: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "


Trace:  (0.848054140154536+0j)
Trace:  (1+0j)
Fidelity of the replied ideal quantum state:  1.0000014245191027
Diamond norm btw ideal choi and ideal estimated channel:  5.0474483950527275e-08
Fidelity of the replied lasso estimated quantum state:  1.0216162211699902
Diamond norm btw ideal choi and approx lasso estimated channel:  0.03413750547263072


In [ ]:
choi_lasso = 2**nqubits*solution_lasso

Not bad result, but it takes too long for 3 qubits. Maybe we can use it as a benchmark to characterize a quantum channel with noise, compare it to the ideal one, and apply corrections so that they look the same.

In [ ]:
#im leaving here the DS approach that does not work at the moment
def invers_map(vec):
    suma=0
    for i in range(len(randoms_PA)):
        for j in range(len(selected_PB)):
            suma += vec[len(selected_PB)*i + j]*cp.kron(randoms_PA[i], selected_PB[j])
    return suma

X_estimated = cp.Variable(shape=((2**nqubits)**2, (2**nqubits)**2))   #Fuck, hay que especificar que es hermítica
cost = cp.norm(X_estimated,'nuc')
constraints = []
vec=[]
for i in range(len(randoms_PA)):
    for j in range(len(selected_PB)):
        vec.append(cp.trace(cp.kron(randoms_PA[i], selected_PB[j]) @ X_estimated) - cp.sqrt(2**nqubits/(len(selected_PB)*len(randoms_PA)))*A_estimated[len(selected_PB)*i + j])

constraints += [cp.norm(invers_map(vec)) <= 1000 ]

constraints += [X_estimated >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_estimated = X_estimated.value

In [741]:
choi = qibo.quantum_info.to_choi(unit_matrix, order='row')
print(diamond_norm(choi, 2**nqubits*solution_estimated))
choi = qibo.quantum_info.to_choi(unit_matrix, order='row')
print(diamond_norm(choi, 2**nqubits*solution_ideal))

1.0011283780537021
4.668956813759951e-09


Here comes the same as before but with quantum states instead of choi maps.

In [305]:
#Let's work first with the ideal values.
nqubits=4
#m = 2**nqubits*nqubits**2
m =  (2**nqubits)*(nqubits**2)
print(m)

c = pseudorandom_circuit(nqubits, density_matrix=True)
randoms_PA = []

A_ideal = []
A_estimated = []

for k in range(m):
    PA = chose_random_Pauli(nqubits)
    u = unitary_transformation_to_computational_basis(PA)
    randoms_PA.append(PA)
    #ideal trace values
    copi_ideal = c.copy()
    density = copi_ideal()
    density = np.array(density)
    trace_val_ideal = np.trace(PA @ density)    #posible fuente de errores si no estoy cogiendo bien el eigenvalue
    #executing circuit
    copi_estimated = c.copy()
    copi_estimated.add(gates.Unitary(u, *range(nqubits)))
    copi_estimated.add(gates.M(*range(nqubits)))
    result = copi_estimated(nshots=1000)
    freq = result.frequencies(binary=True)
    trace_val_estimated = exp_val_in_Z(freq)

    A_ideal.append(trace_val_ideal.real)
    A_estimated.append(trace_val_estimated.real)

print('Ideal trace values', A_ideal)
print('Estimated trace values', A_estimated)

#convex opt problem
X_ideal = cp.Variable(shape=((2**nqubits), (2**nqubits)), hermitian = True)
cost = cp.norm(X_ideal,'nuc')
constraints = []
for i in range(len(randoms_PA)):
        constraints += [cp.trace(randoms_PA[i]@ X_ideal) == A_ideal[i]]

constraints += [X_ideal >> 0]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_ideal = X_ideal.value
print('Done!!')
#print('Diam norm of choi: ', diamond_norm(choi))
#print('Diam norm of solution: ', diamond_norm(solution))
#print('Diamond norm: ', diamond_norm(choi, solution))


X_lasso = cp.Variable(shape=((2**nqubits), (2**nqubits)), hermitian = True)       #tanto complex como hermitica parece que funcionan.
vec=[]
mu = 0.4
norm = np.sqrt(2**nqubits/m)
for i in range(len(randoms_PA)):
        vec.append(cp.trace(randoms_PA[i]@ X_lasso) - A_estimated[i])   #funciona igual con que sin norm
vec = cp.vstack(vec)
cost = 0.5*cp.norm(vec,2)**2 + mu*cp.norm(X_lasso, 'nuc') 
constraints = []

constraints += [X_lasso >> 0]
#constraints += [cp.trace(X_lasso) == 1]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_lasso = X_lasso.value
print('Done!!')


print('Fidelity of the replied ideal quantum state: ', fidelity(density, solution_ideal))
#print('Fidelity of the replied DS quantum state: ', fidelity(density, solution_estimated/np.trace(solution_estimated)))
print('Fidelity of the replied Lasso quantum state: ', fidelity(density, solution_lasso/np.trace(solution_lasso).real))
#print('Trace of the DS: ',np.trace(solution_estimated))
print('Trace of the Lasso: ',np.trace(solution_lasso).real)

256
Ideal trace values [0.030359672207783767, 0.28338931986510496, -0.041198601209760224, -0.040639131696500366, 0.018502930291649865, -0.1636109523753333, -0.040639131696500366, -0.25058809212177974, 0.20724410668994256, -0.019323205190908403, -0.024612658388070578, 0.15258542378102957, 0.08049008490652118, 0.3224297091192527, 0.10915392012776544, -0.18799737688019053, -0.08559633782236908, -0.1636109523753333, 0.2653288706122909, 0.356561402317543, -0.06940446547284165, 0.11538558698866105, -0.009279490662957135, -0.4323394910800611, 0.2653288706122909, 0.14074924939887876, 0.0869081526163296, 0.35177019883620453, -0.1134384513603067, 0.07185648346374501, 0.35177019883620453, -0.4066487630510385, -0.013997793500125458, 0.10158868496269641, 0.07013488444949656, 0.33157111193840916, -0.4533793813691155, 0.06953996956742614, 0.7451777816935968, 0.44369050780724184, -0.12503872212327524, 0.4001665355661645, 0.26125514608637956, -0.21383352563296698, 0.15371685041401462, -0.05593828858105

In [ ]:
#im leving here the expression I used for the DS minimization.
def invers_map(vec):
    suma=0
    for i in range(len(randoms_PA)):
        suma += vec[i]*randoms_PA[i]
    return norm*suma

X_estimated = cp.Variable(shape=((2**nqubits), (2**nqubits)), hermitian = True)
cost = cp.norm(X_estimated,'nuc')
constraints = []
vec = []
norm = np.sqrt(2**nqubits/m)
for i in range(len(randoms_PA)):
        vec.append(norm*cp.trace(randoms_PA[i]@ X_estimated) - norm*A_estimated[i])


constraints += [cp.norm(invers_map(vec)) <= 2 ]

constraints += [X_estimated >> 0]
#constraints += [cp.trace(X_estimated) == 1]

obj = cp.Minimize(cost)
prob = cp.Problem(obj,constraints)
opt_val = prob.solve()
solution_estimated = X_estimated.value
print('Done!!')

In [187]:
print(density)
print(solution_lasso/np.trace(solution_lasso))

[[ 0.1681245 +1.38777878e-17j -0.09984408+2.18482579e-01j
  -0.13178495+2.18546237e-01j -0.00426738-1.30411775e-01j]
 [-0.09984408-2.18482579e-01j  0.34321874+0.00000000e+00j
   0.36227019+4.14702700e-02j -0.16693956+8.29932076e-02j]
 [-0.13178495-2.18546237e-01j  0.36227019-4.14702700e-02j
   0.3873899 +4.16333634e-17j -0.1661782 +1.07770903e-01j]
 [-0.00426738+1.30411775e-01j -0.16693956-8.29932076e-02j
  -0.1661782 -1.07770903e-01j  0.10126687-1.38777878e-17j]]
[[ 0.1649232 +0.j         -0.09768964+0.21821078j -0.13232364+0.21272255j
  -0.01059476-0.13296524j]
 [-0.09768964-0.21821078j  0.34662071+0.j          0.35987653+0.04907678j
  -0.16965977+0.0927688j ]
 [-0.13232364-0.21272255j  0.35987653-0.04907678j  0.38057956+0.j
  -0.16301899+0.12034501j]
 [-0.01059476+0.13296524j -0.16965977-0.0927688j  -0.16301899-0.12034501j
   0.10787653+0.j        ]]


In [137]:
print(3*2**nqubits/np.sqrt(10000))
print(4*m/np.sqrt(10000*m))

0.12
0.4


In [792]:
suma=0
for _ in range(100000):
    ran = np.random.rand(2,2)
    suma += ran/np.trace(ran)

print(suma/100000)

[[0.49992106 0.69923373]
 [0.6989793  0.50007894]]


Parece que funciona bien para recrear la choi representation del quantum channel, al menos si cogemos todas las paulis sale exacto. Veremos que tal al coger menos más adelante

We are going to implement the first code for random Paulis, not with the whole ensamble.

In [72]:
#Let's work first with the ideal values.
nqubits=4

c = pseudorandom_circuit(nqubits, density_matrix=True)
density = c()
density = np.array(density)
solution_list = []
for count in range(10):
    Paulis=[]
    ideal_val = []
    for _ in range(50):
        P = chose_random_Pauli(nqubits)
        Paulis.append(P)
        ideal_pauli_trace_val = np.trace(P*density)
        ideal_val.append(ideal_pauli_trace_val.real)

    X = cp.Variable(shape=(2**nqubits, 2**nqubits), hermitian = True)   #Fuck, hay que especificar que es hermítica

    cost = cp.norm(X,'nuc')
    constraints = [
        cp.trace(Paulis[i] @ X) == ideal_val[i] for i in range(len(ideal_val))
    ]
    constraints += [cp.trace(X) == 1]
    constraints += [X >> 0]

    obj = cp.Minimize(cost)
    prob = cp.Problem(obj,constraints)
    opt_val = prob.solve()
    solution = X.value
    solution_list.append(solution)
    print('Fidelity : ', count, fidelity(density, solution))
#print('Final Fidelity : ', fidelity(density, np.mean(solution_list)))

Fidelity :  0 0.9462677893791094
Fidelity :  1 0.9884811486694227
Fidelity :  2 0.9999910839590436
Fidelity :  3 0.6255762911058912
Fidelity :  4 0.520301185413742
Fidelity :  5 0.7885150788160198
Fidelity :  6 0.9975125377130813
Fidelity :  7 0.6676120648909867
Fidelity :  8 0.9999361494909377
Fidelity :  9 0.9118303298613495


In [73]:
snapshot = np.mean(solution_list, axis=0)
print('Fidelity : ', count, fidelity(density, snapshot))

Fidelity :  9 0.8446023659299584


It does not work if used as a snapshot. Now I want to check if it is better to apply this methos in different subsystems in ordet to reconstruct the final state as a combination of product states, that is $\rho_{AB} = \rho_A \otimes \rho_B$. I just found out that this is not necessarily true!!!

In [190]:
#Let's work first with the ideal values.
from qibo.backends import NumpyBackend

NB = NumpyBackend()
nqubits=4

c = pseudorandom_circuit(nqubits, density_matrix=True)
density = c()
density = np.array(density)
all_states=0
for count in range(10):
    #lets comput the measure on the first two qubits
    ideal_val_A=[]
    rho_A = NB.partial_trace_density_matrix(density, [2,3], nqubits)
    Paulis_A=[]
    for _ in range(50):
        P = chose_random_Pauli(int(nqubits/2))
        Paulis_A.append(P)
        ideal_pauli_trace_val = np.trace(P*rho_A)
        ideal_val_A.append(ideal_pauli_trace_val.real)

    ideal_val_B=[]
    rho_B = NB.partial_trace_density_matrix(density, [0,1], nqubits)
    Paulis_B=[]
    for _ in range(50):
        P = chose_random_Pauli(int(nqubits/2))
        Paulis_B.append(P)
        ideal_pauli_trace_val = np.trace(P*rho_B)
        ideal_val_B.append(ideal_pauli_trace_val.real)

    #Let's reconstruc rho_A
    X_A = cp.Variable(shape=(int(2**(nqubits/2)), int(2**(nqubits/2))), hermitian = True)   #Fuck, hay que especificar que es hermítica

    cost = cp.norm(X_A,'nuc')
    constraints = [
        cp.trace(Paulis_A[i] @ X_A) == ideal_val_A[i] for i in range(len(ideal_val_A))
    ]
    constraints += [cp.trace(X_A) == 1]
    constraints += [X_A >> 0]

    obj = cp.Minimize(cost)
    prob = cp.Problem(obj,constraints)
    opt_val = prob.solve()
    solution_A = X_A.value
    #print('Fidelity : ', fidelity(rho_A, solution_A))

    #Let's reconstruc rho_B
    X_B = cp.Variable(shape=(int(2**(nqubits/2)), int(2**(nqubits/2))), hermitian = True)   #Fuck, hay que especificar que es hermítica

    cost = cp.norm(X_B,'nuc')
    constraints = [
        cp.trace(Paulis_B[i] @ X_B) == ideal_val_B[i] for i in range(len(ideal_val_B))
    ]
    constraints += [cp.trace(X_B) == 1]
    constraints += [X_B >> 0]

    obj = cp.Minimize(cost)
    prob = cp.Problem(obj,constraints)
    opt_val = prob.solve()
    solution_B = X_B.value
    #print('Fidelity : ', fidelity(rho_B, solution_B))      #porque diantres no funciona??


    val_estA, vec_estA = np.linalg.eigh(solution_A)
    val_origA, vec_origA = np.linalg.eigh(rho_A)
    val_estB, vec_estB = np.linalg.eigh(solution_B)
    val_origB, vec_origB = np.linalg.eigh(rho_B)

    suma = 0
    for i in range(len(val_estA)):
        semi_rhoA = np.outer(vec_estA[:,i], np.conjugate(vec_estA[:,i]))
        semi_rhoB = np.outer(vec_estB[:,i], np.conjugate(vec_estB[:,i]))
        suma += val_estA[i]*np.kron(semi_rhoA, semi_rhoB)
    print(count, fidelity(suma, density))
    all_states += suma

print('This is the end: ', fidelity(all_states/100, density))


0 0.6737219415948053
1 0.6734483001965395
2 0.6737219330497473
3 0.6737219648547808
4 0.673721934835586
5 0.6737222366079525
6 0.6737219650358365
7 0.673721159939523
8 0.6737232567612879
9 0.6716448221233349
This is the end:  0.06734869514999392


In [144]:
def reconstruct(eig_val, eig_vec):
    suma=0
    for i in range(len(eig_val)):
        suma += eig_val[i]*np.outer(eig_vec[:,i], np.conjugate(eig_vec[:,i]))
    return suma